In [ ]:
import scdrs
import scanpy as sc
sc.set_figure_params(dpi=300)
from anndata import AnnData
from scipy import stats
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings("ignore")
import llvmlite
print(llvmlite.__version__)
import importlib.metadata
print("importlib.metadata version:", importlib.metadata.version('importlib.metadata'))

In [ ]:
os.chdir("/home/lfzhang/SingleCell/py/scDRS")
adata = sc.read_h5ad("anndata_mdd.h5ad")

In [ ]:
!scdrs munge-gs output.gs --zscore-file eqtl.txt --n-max 1000

In [ ]:
!scdrs compute-score \
    --h5ad_file anndata_mdd.h5ad\
    --h5ad_species human\
    --gs-file processed_geneset.gs\
    --gs-species human\
    --flag-filter-data True\
    --flag-raw-count True\
    --n-ctrl 1000\
    --flag-return-ctrl-raw-score False\
    --flag-return-ctrl-norm-score True\
    --out_folder /home/lfzhang/SingleCell/py/scDRS/results_mdd

In [ ]:
df_gs = pd.read_csv("processed_geneset.gs", sep="\t", index_col=0)
print(df_gs.index)
dict_score = {
    trait: pd.read_csv(f"results_mdd/{trait}.full_score.gz", sep="\t", index_col=0)
    for trait in df_gs.index
}
for trait, df in dict_score.items():
    print(f"Trait: {trait}")
    print(df.head())

In [ ]:
for trait in dict_score:
    adata.obs[trait] = dict_score[trait]["norm_score"]

sc.set_figure_params(figsize=[4, 4], dpi=300)
sc.pl.umap(
    adata,
    color=list(dict_score.keys()),
    color_map="magma",  # 选择一个不同的颜色映射
    vmin=0,  # 调整颜色范围
    vmax=4,
    s=30,  # 调整点大小
    ncols=2,  # 将性状分别绘制在不同列中
    save="MDD_scDRS_mdd.png"
)

In [ ]:
!python compute_downstream.py \
    --h5ad_file anndata_mdd.h5ad \
    --score_file results_mdd/MDD-GWAS.full_score.gz \
    --cell_type "MainType" \
    --flag_gene True\
    --flag_filter False\
    --flag_raw_count False\
    --out_folder /home/lfzhang/SingleCell/py/scDRS/results_mdd

In [ ]:
output_folder = "/home/lfzhang/SingleCell/py/scDRS/results_mdd"
ct_file = f"{output_folder}/MDD.scdrs_ct.MainType"
ct_data = pd.read_csv(ct_file, sep="\t")
print("celltypes_results:")
print(ct_data.head(10))
ct_data.to_csv("/home/lfzhang/SingleCell/py/scDRS/results_mdd/celltype.csv", index=False)
gene_file = f"{output_folder}/MDD.scdrs_gene"
gene_data = pd.read_csv(gene_file, sep="\t")
print("prior_gene_results:")
print(gene_data.head(100))
gene_data.to_csv("/home/lfzhang/SingleCell/py/scDRS/results_mdd/riskgene.csv", index=False)